Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Azure ML Environment Setup

This notebook shows how to setup a Virtual Network to be used for the communication between Ray head and worker nodes in a multi-node cluster setup. In Azure ML, Ray head and worker nodes are run on Azure ML Compute Targets. For the compute targets to be able to communicate to each other, it is necessary to have this virtual network resource properly setup in your Azure ML resource group.

This notebook is based on the example provided [here](https://github.com/Azure/MachineLearningNotebooks/blob/master/how-to-use-azureml/reinforcement-learning/setup/devenv_setup.ipynb).

Azure ML SDK version. All notebooks for this lab were tested with the AML SDK in the version below.

In [1]:
import azureml.core

print("Azure Machine Learning SDK Version: ", azureml.core.VERSION)

Azure Machine Learning SDK Version:  1.17.0


Get a reference to your Azure ML Workspace.

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()

It is assumed that you are running this notebook and all other notebooks for this lab in an Azure ML Compute Instance, in which case the *azure-mgmt-network* package is already available.

In [3]:
# If you need to install the Azure Networking SDK, uncomment the following line.
#!pip install --upgrade azure-mgmt-network

Create the Virtual Network resource.

In [4]:
from azure.mgmt.network import NetworkManagementClient

# Virtual network name
vnet_name ="your_vnet"

# Default subnet
subnet_name ="default"

# The Azure subscription you are using
subscription_id=ws.subscription_id

# The resource group for the reinforcement learning cluster
resource_group=ws.resource_group

# Azure region of the resource group
location=ws.location

network_client = NetworkManagementClient(ws._auth_object, subscription_id)

async_vnet_creation = network_client.virtual_networks.create_or_update(
    resource_group,
    vnet_name,
    {
        'location': location,
        'address_space': {
            'address_prefixes': ['10.0.0.0/16']
        }
    }
)

async_vnet_creation.wait()
print("Virtual network created successfully: ", async_vnet_creation.result())

Virtual network created successfully:  {'additional_properties': {}, 'id': '/subscriptions/a6c2a7cc-d67e-4a1a-b765-983f08c0423a/resourceGroups/alvilcek-ml-rg2/providers/Microsoft.Network/virtualNetworks/your_vnet', 'name': 'your_vnet', 'type': 'Microsoft.Network/virtualNetworks', 'location': 'eastus', 'tags': None, 'address_space': <azure.mgmt.network.v2020_06_01.models._models_py3.AddressSpace object at 0x7fb0e59b87f0>, 'dhcp_options': None, 'subnets': [], 'virtual_network_peerings': [], 'resource_guid': '754aab43-22b3-49cc-b5d8-80fe12fd49bc', 'provisioning_state': 'Succeeded', 'enable_ddos_protection': False, 'enable_vm_protection': False, 'ddos_protection_plan': None, 'bgp_communities': None, 'ip_allocations': None, 'etag': 'W/"401beee2-eb8a-4e11-b55a-9d3a5b108242"'}


Setup a Network Security Group resource to open the necessary ports for the Ray cluster communication.

In [5]:
import azure.mgmt.network.models

security_group_name = vnet_name + '-' + "nsg"
security_rule_name = "AllowAML"

# Create a network security group
nsg_params = azure.mgmt.network.models.NetworkSecurityGroup(
    location=location,
    security_rules=[
        azure.mgmt.network.models.SecurityRule(
            name=security_rule_name,
            access=azure.mgmt.network.models.SecurityRuleAccess.allow,
            description='Reinforcement Learning in Azure Machine Learning rule',
            destination_address_prefix='*',
            destination_port_range='29876-29877',
            direction=azure.mgmt.network.models.SecurityRuleDirection.inbound,
            priority=400,
            protocol=azure.mgmt.network.models.SecurityRuleProtocol.tcp,
            source_address_prefix='BatchNodeManagement',
            source_port_range='*'
        ),
    ],
)

async_nsg_creation = network_client.network_security_groups.create_or_update(
    resource_group,
    security_group_name,
    nsg_params,
)

async_nsg_creation.wait() 
print("Network security group created successfully:", async_nsg_creation.result())

network_security_group = network_client.network_security_groups.get(
    resource_group,
    security_group_name,
)

# Define a subnet to be created with network security group
subnet = azure.mgmt.network.models.Subnet(
            id='default',
            address_prefix='10.0.0.0/24',
            network_security_group=network_security_group
            )
    
# Create subnet on virtual network
async_subnet_creation = network_client.subnets.create_or_update(
    resource_group_name=resource_group,
    virtual_network_name=vnet_name,
    subnet_name=subnet_name,
    subnet_parameters=subnet
)

async_subnet_creation.wait()
print("Subnet created successfully:", async_subnet_creation.result())

Network security group created successfully: {'additional_properties': {}, 'id': '/subscriptions/a6c2a7cc-d67e-4a1a-b765-983f08c0423a/resourceGroups/alvilcek-ml-rg2/providers/Microsoft.Network/networkSecurityGroups/your_vnet-nsg', 'name': 'your_vnet-nsg', 'type': 'Microsoft.Network/networkSecurityGroups', 'location': 'eastus', 'tags': None, 'security_rules': [<azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule object at 0x7fb0e499b0b8>], 'default_security_rules': [<azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule object at 0x7fb0e499b0f0>, <azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule object at 0x7fb0e499b128>, <azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule object at 0x7fb0e499b160>, <azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule object at 0x7fb0e499b198>, <azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule object at 0x7fb0e499b1d0>, <azure.mgmt.network.v2020_06_01.models._models_py3.SecurityRule 

Review the virtual network security rules.

In [6]:
import sys, os  
sys.path.insert(0, os.path.join(os.getcwd(), './utils'))

from networkutils import *

check_vnet_security_rules(ws._auth_object, ws.subscription_id, ws.resource_group, vnet_name, True)

INFORMATION: Rule matched with required ports. Subnet: default Rule: AllowAML
INFORMATION: Network security group rules for your virtual network are saved in file your_vnet.csv
